<h2 align="center">Codebasics DL Course: Vehicle Damage Detection Project: Hyperparameter Tunning</h2>

In [55]:
import os
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import time
import torchvision.models as models
from matplotlib import pyplot as plt
import optuna

In [56]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

# Load Data

In [57]:
image_transforms = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.ColorJitter(brightness=0.2, contrast=0.2),
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

In [58]:
dataset_path = "./dataset"
dataset = datasets.ImageFolder(root=dataset_path, transform = image_transforms)  
len(dataset)                

2300

In [59]:
2300*0.75

1725.0

In [60]:
class_names = dataset.classes
class_names

['F_Breakage', 'F_Crushed', 'F_Normal', 'R_Breakage', 'R_Crushed', 'R_Normal']

In [61]:
num_classes = len(dataset.classes)
num_classes

6

In [62]:
train_size = int(0.75*len(dataset))
val_size = len(dataset) - train_size

train_size, val_size 

(1725, 575)

In [63]:
from torch.utils.data import random_split 
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

In [64]:
train_loader = DataLoader(train_dataset, batch_size = 32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=True)

## Model Trainig & Hyper Parameter Tuning

In [65]:
# Load the pre-trained ResNet Model 
class CarClassifierResNet(nn.Module):
    def __init__(self, num_classes, dropout_rate=0.5):
        super().__init__()
        self.model = models.resnet50(weights='DEFAULT')
       
        # Freeze all layers except the final fully connected layer
        for param in self.model.parameters():
            param.requires_grad = False

        # Unfreeze layer4 and fc layers
        for param in self.model.layer4.parameters():
            param.requires_grad = True

        # Replace the final fully connected layer
        self.model.fc = nn.Sequential(
            nn.Dropout(dropout_rate),
            nn.Linear(self.model.fc.in_features, num_classes)
        )
        
    def forward(self,x):
        x = self.model(x)
        return x 

In [68]:
# Define the objective function for Optuna 
def objective(trial):
    # Suggest values for the hyperparameters 
    lr = trial.suggest_float('lr', 1e-5, 1e-2, log=True)
    dropout_rate = trial.suggest_float('dropout_rate', 0.2, 0.7)

    # Load the model 
    model = CarClassifierResNet(num_classes = num_classes, dropout_rate = dropout_rate).to(device)

    # Define the loss function and optimizer
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), lr=lr)

    # Training loop(using fewaerr epochs for faaster hyperparameter tuning)
    epochs = 3
    start = time.time()
    for epoch in range(epochs):
        model.train()
        running_loss = 0.0
        for batch_num, (images, labels) in enumerate(train_loader):
            images, labels = images.to(device), labels.to(device)

            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()*images.size(0)

        epoch_loss = running_loss/len(train_loader.dataset)

        # Validataion loop
        model.eval()
        correct = 0
        total = 0
        with torch.no_grad():
            for images,labels in val_loader:
                images, labels = images.to(device), labels.to(device)
                outputs = model(images)
                _, predicted = torch.max(outputs.data, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()
        accuracy = 100*correct/total

        # Report intermediate result to Optuna 
        trial.report(accuracy, epoch)

        # Handle pruning (if applicable)
        if trial.should_prune():
            raise optuna.exceptions.TrialPruned()

    end = time.time()
    print(f"Execution time: {end-start} seconds")

    return accuracy

In [69]:
# Create the study and opimize
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=20)

[I 2025-03-20 22:55:27,190] A new study created in memory with name: no-name-3f25d52a-7167-4915-9e75-1fd180a84084
[I 2025-03-20 23:04:36,143] Trial 0 finished with value: 74.95652173913044 and parameters: {'lr': 0.00010125520081040568, 'dropout_rate': 0.28929877070153254}. Best is trial 0 with value: 74.95652173913044.


Execution time: 548.1944437026978 seconds


[I 2025-03-20 23:16:07,192] Trial 1 finished with value: 77.04347826086956 and parameters: {'lr': 0.0008135655593386685, 'dropout_rate': 0.6042468473122996}. Best is trial 1 with value: 77.04347826086956.


Execution time: 690.2759158611298 seconds


[I 2025-03-20 23:27:51,298] Trial 2 finished with value: 50.608695652173914 and parameters: {'lr': 1.3721193611533408e-05, 'dropout_rate': 0.6399435710259243}. Best is trial 1 with value: 77.04347826086956.


...

[I 2025-03-21 01:04:16,115] Trial 16 finished with value: 79.65217391304348 and parameters: {'lr': 0.0003211633591360054, 'dropout_rate': 0.34889421890576205}. Best is trial 16 with value: 79.65217391304348.


Execution time: 466.68794107437134 seconds


[I 2025-03-21 01:06:51,465] Trial 17 pruned. 
[I 2025-03-21 01:14:43,746] Trial 18 finished with value: 77.56521739130434 and parameters: {'lr': 0.0003168241021247859, 'dropout_rate': 0.4156965883473651}. Best is trial 16 with value: 79.65217391304348.


Execution time: 471.72010469436646 seconds


[I 2025-03-21 01:19:53,822] Trial 19 pruned. 


In [70]:
study.best_params

{'lr': 0.0003211633591360054, 'dropout_rate': 0.34889421890576205}